<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🔨👨‍⚖️ תהליך הסבת שכבת גבולות פסקי דין לבנק"ל מודרני 🔨👨‍⚖️ 

In [ ]:
print(' Judgments borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import datetime as dt
import geopandas as gpd
from configs import CNFG
from helpers import assign_BlockName

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות פס"ד קיימת

In [ ]:
GVUL_PSAK_DIN_cols =  ['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'NAME', 'SOURCE', 'GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'STATUS_DATE', 'geometry']

GVUL_PSAK_DIN = gpd.read_file(CNFG.CADSDE + 'GVUL_PSAK_DIN_NEW1.gdb', layer = 'GVUL_PSAK_DIN_NEW1', include_fields = GVUL_PSAK_DIN_cols)

del GVUL_PSAK_DIN_cols

<h1 style="text-align:right;font-size:100%">
טבלת פס"ד

In [ ]:
orcale_cols = ['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'NAME', 'SOURCE', 'GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'STATUS_DATE']

oracle_df = pd.read_csv(CNFG.Oracle + 'psak_din.csv', encoding = 'cp1255', usecols = orcale_cols)

del orcale_cols

<h1 style="text-align:right;font-size:100%">
רשימת פס"ד שיש להתעלם מהם

In [ ]:
JudgmentToIgnore = pd.read_excel(CNFG.Local + 'JudgmentsToIgnore.xlsx', usecols = ['ProcessName']).query('ProcessName.notna()')

<h1 style="text-align:right;font-size:100%">
שכבת גושים מודרנית

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'GlobalID', 'IsTax'])
Blocks = Blocks[Blocks['IsTax'] == 0].drop(columns = 'IsTax')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('   Cleaning data')

<h1 style="text-align:right;font-size:125%">
:עיבוד שכבת גבולות פס"ד
<h1 style="text-align:right;font-size:100%">
הוספת שדה של שם התכנית לפי מספר ושנת פס"ד

In [ ]:
GVUL_PSAK_DIN[['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR']] = GVUL_PSAK_DIN[['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR']].astype(int)

GVUL_PSAK_DIN['ProcessName'] = GVUL_PSAK_DIN['PSAK_DIN_NUM'].astype(str) + '-' + GVUL_PSAK_DIN['PSAK_DIN_YEAR'].astype(str)
GVUL_PSAK_DIN.drop(columns = ['PSAK_DIN_NUM', 'PSAK_DIN_YEAR'], inplace=True)

GVUL_PSAK_DIN = assign_BlockName(GVUL_PSAK_DIN, 'GUSH_NUM', 'GUSH_SUFFIX')

GVUL_PSAK_DIN['STATUS'] = GVUL_PSAK_DIN['STATUS'].astype('Int64')

<h1 style="text-align:right;font-size:125%">
:עיבוד טבלת פס"ד
<h1 style="text-align:right;font-size:100%">
הוספת שדה של שם התכנית לפי מספר ושנת פס"ד
<h1 style="text-align:right;font-size:100%">
הגדרת שדות של תאריך כשדות מסוג תאריך
<h1 style="text-align:right;font-size:100%">
הגדרת שדה למספר מלא לגוש לצורך מיזוג בהמשך
<h1 style="text-align:right;font-size:100%">
הסרת פסקי דין בהתאם לרשימה של פסקי דין להסרה

In [ ]:
oracle_df[['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR']] = oracle_df[['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR']].astype(int)
oracle_df['ProcessName'] = oracle_df['PSAK_DIN_NUM'].astype('str') + '-' + oracle_df['PSAK_DIN_YEAR'].astype('str')
oracle_df = oracle_df.drop(columns = ['PSAK_DIN_NUM', 'PSAK_DIN_YEAR'])

oracle_df = assign_BlockName(oracle_df, 'GUSH_NUM', 'GUSH_SUFFIX')

oracle_df['STATUS_DATE'] = pd.to_datetime(oracle_df['STATUS_DATE'], format = '%d/%m/%Y %H:%M:%S')

oracle_df['STATUS'] = oracle_df['STATUS'].astype('Int64')

In [ ]:
print('   Data conversion')

<h1 style="text-align:right;font-size:125%">
מיזוג טבלאות 

In [ ]:
collection = pd.concat([GVUL_PSAK_DIN, oracle_df], ignore_index = True)\
              .sort_values('ProcessName')

aggfunc = {'PSAK_DIN_ID' : 'first', 'STATUS' : 'max', 'BlockName' : 'first', 'NAME': 'first', 'SOURCE' : 'max', 'STATUS_DATE' : 'max'}
collection = collection.dissolve(by = 'ProcessName', as_index = False, aggfunc = aggfunc)

collection = collection[~collection['ProcessName'].isin(JudgmentToIgnore['ProcessName'])]

del [GVUL_PSAK_DIN, oracle_df, aggfunc]

In [ ]:
collection = collection.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')\
                       .rename(columns = {'GlobalID' : 'BlockUniqueID'})

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('   Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("geometry", gpd.array.GeometryDtype),       
                       #("GlobalID", str),
                        ("ProcessName", str),
                        ("ProcessType", int),
                        ("Status", int),
                        ("BlockUniqueID", str),
                        ("GeodeticNetwork", int),
                        ("SurveyorLicenseID", int),
                        ("DataSource", int),
                        ("PlanName", str),  
                        ("ORACLE_ID", int),
                       #("Shape_Length", float),
                       #("Shape_Area", float),
                        ])


Judgments = gpd.GeoDataFrame(np.empty(0, dtype = new_columns),
                             crs = 2039,
                             geometry = 'geometry')

del new_columns

<h1 style="text-align:right;font-size:100%">
    איכלוס שדות

In [ ]:
Judgments['BlockUniqueID'] = collection['BlockUniqueID']


Judgments['ProcessType'] = 3 # פסק דין


Judgments['ProcessName'] = collection['ProcessName'].astype(str)


statusDict = {1 : 11,    #בטיפול האגף - בטיפול האגף
              2 : 12,    #בבדיקת הממונה על המרשם - בבדיקת הממונה על המרשם
              3 : 13,    # מאושר - מאושר
              4 : 9,    # מבוטל - מבוטל
              None : 9} #ללא ערך - מבוטל
Judgments['Status'] = collection['STATUS'].map(statusDict)
Judgments['Status'] = Judgments['Status'].fillna(36)
Judgments['Status'] = Judgments['Status'].astype(int)


Judgments['PlanName'] = collection['NAME']


Judgments['GeodeticNetwork'] = 0  #רשת לא ידועה


#Judgments['SurveyorLicenseID'] = None


Judgments['DataSource'] = 5 #CAD


Judgments['geometry'] = collection['geometry']


Judgments['ORACLE_ID'] = collection['PSAK_DIN_ID']

In [ ]:
Judgments.sort_values('ORACLE_ID', inplace=True)
del collection

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('   Appending results to CadasterProcessBorders')

In [ ]:
Polygons      = Judgments[(~Judgments['geometry'].isna()) & (Judgments['geometry'].geom_type == 'Polygon')]

MultiPolygons = Judgments[(~Judgments['geometry'].isna()) & (Judgments['geometry'].geom_type == 'MultiPolygon')]

EmptyGeometry = Judgments[Judgments['geometry'].isna()]

In [ ]:
Polygons.to_file(CNFG.Results, layer = 'PT3', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')
MultiPolygons.to_file(CNFG.Results, layer = 'PT3', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'a')
EmptyGeometry.to_file(CNFG.Results, layer = 'PT3', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'a')